In [1]:
#classe image
import os
from PIL import Image
from PIL import ImageEnhance, ImageFilter,  ImageOps
from PIL.ImageFilter import MedianFilter


#Création d'une classe image
class CustomImage :
    def __init__(self,path=0,folder="transformed_image"):
            self.image = Image.open(path) #retient l'image
            self.image.thumbnail((600,300)) #change la taille de l'image pour être adaptée à la fenêtre de l'application
            self.width,self.height = self.image.size #largeur et hauteur de l'image
            self.path = path #chemin de l'image
            self.format = self.image.format # format de l'image
            self.nom, self.extension = os.path.splitext(path) #permet de separer le nom de l'image et son extension

    #fonction copie qui prend en argument une CustomImage et en revoie une copie
    def copie(self):
        image_mod = CustomImage(self.path) # nouvelle CustomImage
        image_mod.image = self.image
        image_mod.width = self.width 
        image_mod.height = self.height
        image_mod.path = self.path
        image_mod.format = self.format
        image_mod.nom = self.nom
        image_mod.extension = self.extension
        
        return image_mod #renvoi de la nouvelle CustomImage ayant les mêmes propriétés que celle en argument 
    
    #fonction zoom qui permet de reduire ou augmenter la taille d'une image
    #prend en argument une CustomImage et renvoie une CustomImage dont la taille a été modifiée
    def Zoom(self,size=0.5,width_max = 1500, height_max = 1000, quality=75):
        new_width = round(self.width*size) #pour arrondir
        new_height = round(self.height * size)

        #Cas où la taille de l'image dépasse la taille max
        if (new_width > width_max) or (new_height > height_max) :
            
            
            if new_width > new_height :
                facteur = width_max/new_width
                new_width = width_max 
                new_height = round(new_height*facteur) 
                
            else :
                facteur = height_max/new_height
                new_height = height_max 
                new_width = round(new_width*facteur)
                
        self.image = self.image.resize((new_width,new_height),Image.ANTIALIAS) #antialias pour avoir une meilleure qualité
        return self
        
    #fonction rotation vers la droite qui permet de faire pivoter une image vers la droite
    #prend en argument une CustomImage et renvoie la CustomImage transformée
    def RightRotation(self,angle = 270,quality=75):
        
        if angle == 90 :
            self.image = self.image.transpose(Image.ROTATE_90)
        if angle == 180 :
            self.image = self.image.transpose(Image.ROTATE_180)
        if angle == 270 :
            self.image = self.image.transpose(Image.ROTATE_270) 
        
        #on inverse la largeur et la hauteur de l'image pour garder les bonnes mesures
        w = self.height
        h = self.width
        self.height = h
        self.width = w
        
        return self
        
    
    # fonction contraste : permet d'augmenter ou de diminuer le contraste de la CustomImage
    #prend en argument une CustomImage et renvoie la CustomImage modifiée 
    def Contrast(self,scale = 1,quality=75):
        scale_value=scale 
        self.image = ImageEnhance.Contrast(self.image).enhance(scale) #fonction de PIL permettant le changement de contaste
        return self

    # fonction flou : permet de flouter la CustomImage (aussi appelé filtre gaussien)
    #prend en argument une CustomImage et renvoie la CustomImage modifiée
    def Fuzzy(self,scale = 2, quality=75):
        self.image = self.image.filter(ImageFilter.GaussianBlur(radius= scale))
        return self
        
    # fonction négatif : permet de passer en négatif la CustomImage 
    #prend en argument une CustomImage et renvoie la CustomImage modifiée
    def Negative(self,quality=75):
        self.image = ImageOps.invert(self.image)
        return self

    # fonction sobel ou détection des contours  : permet d'appliquer la detection des contours à la CustomImage 
    #prend en argument une CustomImage et renvoie la CustomImage modifiée
    def Sobel(self,quality=75):
        self.image = self.image.filter(ImageFilter.FIND_EDGES)
        return self
    
    # fonction médiane : permet d'appliquer le filtre médian à la CustomImage 
    #prend en argument une CustomImage et renvoie la CustomImage modifiée
    def Median(self,quality = 75):
        self.image = self.image.filter(MedianFilter())
        return self

    # fonction lissage ou moyenne : permet de lisser la CustomImage 
    #prend en argument une CustomImage et renvoie la CustomImage modifiée
    def Moyen(self,quality=75):
        self.image = self.image.filter(ImageFilter.SMOOTH)
        return self
        
    # fonction niveaux de gris : permet de passer la CustomImage en niveaux de grix
    #prend en argument une CustomImage et renvoie la CustomImage modifiée
    def GrayScale(self,quality=75):
        self.image = self.image.convert('L')
        return self
    
    # fonction netteté : permet d'augmenter la netteté de la CustomImage 
    #prend en argument une CustomImage et renvoie la CustomImage modifiée
    def Sharpen(self,value=1,quality=75):
        self.image = self.image.filter(ImageFilter.UnsharpMask(radius=value))
        return self
    

In [2]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QGridLayout, QPushButton, QLineEdit, QSlider, QLabel, QHBoxLayout, QVBoxLayout, QGraphicsAnchorLayout, QComboBox 
from PyQt5.Qt import *
from PIL.ImageQt import ImageQt
from PyQt5 import QtGui

#class permettant de transformer une image en bouton
class PicButton(QAbstractButton):
    def __init__(self, pixmap, parent=None):
        super(PicButton, self).__init__(parent)
        self.pixmap = pixmap.scaled(150, 150, Qt.KeepAspectRatio)
    
    #procédure permettant l'affichage du bouton-image
    def paintEvent(self, event):
        painter = QPainter(self)
        painter.drawPixmap(event.rect(), self.pixmap)
    
    #procédure permettant de changer l'image du bouton-image déjà existant
    def changePic(self, pic):
        self.pixmap = pic.scaled(500, 300, Qt.KeepAspectRatio)
    
    #fonction retournant la taille de l'image
    def sizeHint(self):
        return self.pixmap.size()


#class permettant de transformer une image PIL en QPixmap (utilisable avec Qt)
def pil2pixmap(im):
    if im.mode == "RGB":
        r, g, b = im.split()
        im = Image.merge("RGB", (b, g, r))
    elif  im.mode == "RGBA":
        r, g, b, a = im.split()
        im = Image.merge("RGBA", (b, g, r, a))
    elif im.mode == "L":
        im = im.convert("RGBA")
    im2 = im.convert("RGBA")
    data = im2.tobytes("raw", "RGBA")
    qim = QtGui.QImage(data, im.size[0], im.size[1], QtGui.QImage.Format_ARGB32)
    pixmap = QtGui.QPixmap.fromImage(qim)
    return pixmap

In [3]:
from PyQt5.QtGui import *

#fenetre de dialogue pour l'enregistrement de l'image une fois les modifications finies
class FenetreEnregistre(QDialog):
    def __init__(self, image, parent=None):
        super().__init__(parent=parent)
        
        self.setWindowTitle("Enregistrer") #le titre de la fenetre est mis a 'Enregistrer'
        self.setWindowIcon(QIcon("disquette.jpg")) # l'icone de la fenetre devient une disquette
        self.resize(750,400) #on ajuste la taille de la fenêtre
        self.image = image #la valeur self.image est initialisé avec la valeur image qui correspond à l'image à enregister
        
        #on crée la première ligne du formulaire à remplir : le nom donnée à la nouvelle image
        l1 = QLabel("Nom") #titre de la ligne
        self.nom = QLineEdit() #lineEdit permettant à l'utilisateur d'entrer le nom voulu
    
        # on crée la deuxième ligne du formulaire ) remplir : le chemin dans lequel doit être enregistrer l'image
        l2 = QLabel("Chemin") #titre de la ligne
        self.chemin = QLineEdit() #lineEdit permettant à l'utilisateur d'entrer le nom voulu
        
        #on crée la troisième ligne du formulaire : un menu déroulant pour selectionner le format dans lequel doit être enregister l'image
        formatlabel = QLabel("Format") #titre de la ligne
        self.formatimgList = QComboBox() #menu déroulant
        self.formatimgList.addItem("JPG") #première option du menu deroulant : jpeg
        self.formatimgList.addItem("PNG") #deuxième option du menu deroulant : png
        self.formatimgList.addItem("PDF") #troisième option du menu deroulant : pdf
        
        #bouton enregistrer
        self.enregistre = QPushButton("Enregistrer")
        self.enregistre.clicked.connect(self.Enregistre) #connexion du bouton à la fonction qu'il appelle quand il est cliqué
        
        # layout de la fenetre : 
        form = QFormLayout()
        form.addRow(l1,self.nom) # on ajoute la première ligne 
        form.addRow(l2,self.chemin) # on ajoute la deuxième ligne
        form.addRow(formatlabel, self.formatimgList) # on ajoute la troisème ligne
        form.addRow(self.enregistre) # on ajoute le bouton enregistrer
        form.setSpacing(70) # on ajuste l'espace entre les lignes du layout
        self.setLayout(form) #on applique le layout à la fenetre
        
    #fonction liée au bouton enregistrer : permet l'enregistrement de l'image 
    def Enregistre(self):
        nom = self.nom.text() #on recupère le nom de l'image dans le lineEdit correspondant
        chemin = self.chemin.text() # on recupère le chemin de l'image dans le lineEdit correspondant
        formatimgtxt = self.formatimgList.currentText().lower() #on recupère le format de l'image et on la passe en minuscule
        formatimgSave = self.formatimgList.currentText() # on recupère le format de l'image
        #si le format selectionné est le format jpg on change la valeur de formatimagesave pour correspondre aux exigences de PIL
        if formatimgSave == 'JPG':
            formatimgSave='JPEG'
        #si le chemin exite ou il est rester vide on peut enregistrer l'image
        if os.path.exists(chemin) or chemin == '' : 
            cheminImage =  os.path.join(chemin, nom + '.' + formatimgtxt) #on créer le chemin entier de l'image
            self.image.image.save(cheminImage,formatimgSave,quality=70) #on sauvegarde l'image
            self.close() #on ferme la fenetre d'enregistrement
        #si le chemin n'existe pas, on affiche une message d'erreur à l'aide d'un pop-up
        else :
            CheminInvalide = QMessageBox() #création du pop-up
            CheminInvalide.setIcon(QMessageBox.Information) #il est de type information

            CheminInvalide.setText("Chemin Invalide") #message délivré par le pop-up
            CheminInvalide.exec_() #on execute le pop-up pour qu'il s'affiche

In [4]:
#fenetre principale
class Fenetre(QWidget):
    
    #constructeur
    def __init__(self):
        
        QWidget.__init__(self)
        #titre de la fenetre
        self.setWindowTitle("Application Traitement d'images")
        #Logo de la fenetre
        self.setWindowIcon(QIcon("pinceau.png"))
        #ajustement de la taille de la fenêtre
        self.setFixedSize(1700,1050)
        #attribut permettant de savoir si une image a été importée ou non dans la fenetre
        self.imageImportee = 0

        
        #layout de la fenetre principale
        layout1 = QVBoxLayout()
        
        #label 2 : image, chemin, bouton OK
        layout2 = QHBoxLayout()
        #creation d'un label Image et ajout au layout 2
        image = QLabel('Image : ')
        layout2.addWidget(image)
        #creation d'un lineEdit pour recuperer le chemin de l'image a transformer et ajout au layout 2
        self.cheminImage = QLineEdit("Chemin")
        layout2.addWidget(self.cheminImage)
        #creation du bouton OK qui permet d'ouvrir l'image dans la fenetre et ajout au layout 2
        self.btnOk = QPushButton('OK')
        layout2.addWidget(self.btnOk)
        self.btnOk.clicked.connect(self.btn_ok) #connection a la fontcion qui se déclenche quand le bouton est cliqué
        
        
        
        # layout img : caractéristiques de l'image, image, bouton retour, bouton reinitialiser
        layoutimg = QHBoxLayout()
        
        #layoutcaracimg : creation de 3 label -> taille, format, nom de l'image et ajout au layout caracimg
        layoutcaracimg = QVBoxLayout() #création d'un layout vertical
        #label comportant le nom de l'image lorsque importée
        self.nom = QLabel("Nom :")
        self.nom.setFixedSize(200,100) #on fixe la taille du label
        #label comportant le taille de l'image lorsque importée
        self.taille = QLabel("Taille :")
        self.taille.setFixedSize(200,100) #on fixe la taille du label
        #label comportant le nom de l'image lorsque importée
        self.format = QLabel("Format : ")
        self.format.setFixedSize(200,100) #on fixe la taille du label
        #on ajoute tous les labels créés au layout caracimg
        layoutcaracimg.addWidget(self.nom)
        layoutcaracimg.addWidget(self.taille)
        layoutcaracimg.addWidget(self.format)
        layoutcaracimg.setAlignment(Qt.AlignTop) #les widgets sont alignés vers le haut dans le layout 
        
        #creation d'un label cadre qui permettra par la suite l'affichage de l'image a transformer
        self.cadre = QLabel()
        self.cadre.setFixedSize(600,300) #on fixe la taille du label
        self.cadre.setStyleSheet("background-color:gray") #on affiche un rectangle gris lorqu'on ouvre la fenetre pour repérer l'endroit où s'affichera l'image importée
        
        #layout btn : creation de 2 PushButton -> retour et reenitialiser qui permettrons de défaire la dernière action effectuée
        #sur l'image et revenir à l'image de base importée
        layoutimgbtn = QVBoxLayout() #creation d'un layout vertical
        #création du bouton retour
        self.btnRetour = QPushButton("Retour")
        self.btnRetour.setFixedSize(200,70) #on fixe la taille du bouton
        self.btnRetour.clicked.connect(self.btn_Retour) #on connecte le bouton à la fonction qui est appelée lorsqu'il est cliqué
        layoutimgbtn.addWidget(self.btnRetour) #on ajoute le bouton au layout imgbtn
        #création du bouton reinitialiser
        self.btnReinit = QPushButton("Réinitialiser")
        self.btnReinit.setFixedSize(200,70) #on fixe la taille du bouton
        self.btnReinit.clicked.connect(self.btn_Reinit)  #on connecte le bouton à la fonction qui est appelée lorsqu'il est cliqué
        layoutimgbtn.addWidget(self.btnReinit) #on ajoute le bouton au layout imgbtn
        layoutimgbtn.setAlignment(Qt.AlignRight) # les widgets sont alignés à droite dans le layout
        layoutimgbtn.setAlignment(Qt.AlignTop) # les widgets sont alignés vers le haut dans le layout
        
        #ajout des 2 sous layout et du label cadre au layout img
        layoutimg.addLayout(layoutcaracimg)
        layoutimg.addWidget(self.cadre)
        layoutimg.addLayout(layoutimgbtn)
        
        #layout 3 : bouton rotation, curseur zoom, curseur contraste, curseur flou/netteté
        layout3 = QHBoxLayout() #création d'un layout horizontal
        #bouton rotation
        self.btnRotation = QPushButton()
        #on change l'icone du bouton rotation par une image de fleche effectuant une rotation vers la droite
        icon = QIcon("fleche_rotation.jpg") #creation d'une icone 
        self.btnRotation.setIcon(icon) #application au bouton
        self.btnRotation.setFixedSize(150,70) # on fixe la taille du bouton
        self.btnRotation.clicked.connect(self.btn_Rotation)  #on connecte le bouton à la fonction qui est appelée lorsqu'il est cliqué
        layout3.addWidget(self.btnRotation) # on ajoute le bouton au layout3
        
        #curseur zoom
        layoutCZ = QVBoxLayout() #creation du layout vertical pour le curseur de zoom
        self.zoomlabel = QLabel("Zoom") #on crée un label faisant office de titre pour le curseur
        layoutCZ.addWidget(self.zoomlabel) #on ajoute le titre au layout
        layoutCZ2 = QHBoxLayout() #creation d'un nouveau layout horizontal pour le curseur et l'affichage de sa valeur
        self.curseurZoom = QSlider(Qt.Horizontal) #creation du curseur zoom
        self.curseurZoom.setMinimum(0) #on fixe sa valeur minimale a 0
        self.curseurZoom.setMaximum(100) #on fixe sa valeur maximale a 100
        self.curseurZoom.setValue(50) # on fixe sa valuer de départ à 50
        self.curseurZoom.setFixedSize(200,20) #on fixe la taille du curseur
        layoutCZ2.addWidget(self.curseurZoom) #on ajoute le curseur au layout
        #label état du curseur zoom
        self.pourcentZoom = QLabel('50%') #on initialise la valeur du label a 50%
        self.pourcentZoom.setFixedSize(70,40) #on fixe la taille de label
        self.pourcentZoom.setFrameStyle(QFrame.Panel | QFrame.Plain) #on modifie le style d'affiche en lui donnant un contour
        layoutCZ2.addWidget(self.pourcentZoom) #on ajoute le label au layout horizontal
        layoutCZ.addLayout(layoutCZ2) #on ajoute le layout horizontal au layout vertical
        layout3.addLayout(layoutCZ)#on ajoute le layout vertical de zoom au layout 3
        self.curseurZoom.valueChanged.connect(self.valuechangeZoom) #on connecte le curseur à la fonction qui s'applique lorsque le curseur est déplacé
        
        #curseur constraste
        layoutCC = QVBoxLayout()#creation du layout vertical pour le curseur de contraste
        self.contrastelabel = QLabel("Contraste") #on crée un label faisant office de titre pour le curseur
        layoutCC.addWidget(self.contrastelabel)  #on ajoute le titre au layout
        layoutCC2 = QHBoxLayout() #creation d'un nouveau layout horizontal pour le curseur et l'affichage de sa valeur
        self.curseurContraste = QSlider(Qt.Horizontal) #creation du curseur contraste
        self.curseurContraste.setMinimum(0) #on fixe sa valeur minimale a 0
        self.curseurContraste.setMaximum(100) #on fixe sa valeur maximale a 100
        self.curseurContraste.setValue(50) # on fixe sa valuer de départ à 50
        self.curseurContraste.setFixedSize(200,20)  #on fixe la taille du curseur
        layoutCC2.addWidget(self.curseurContraste)  #on ajoute le curseur au layout
        #label etat du curseur contraste 
        self.pourcentContraste = QLabel('50%') #on initialise la valeur du label a 50%
        self.pourcentContraste.setFixedSize(70,40)  #on fixe la taille de label
        self.pourcentContraste.setFrameStyle(QFrame.Panel | QFrame.Plain) #on modifie le style d'affiche en lui donnant un contour
        layoutCC2.addWidget(self.pourcentContraste) #on ajoute le label au layout horizontal
        layoutCC.addLayout(layoutCC2)  #on ajoute le layout horizontal au layout vertical
        layout3.addLayout(layoutCC) #on ajoute le layout vertical de zoom au layout 3
        self.curseurContraste.valueChanged.connect(self.valuechangeContraste) #on connecte le curseur à la fonction qui s'applique lorsque le curseur est déplacé
        
        #curseur flou/netteté
        layoutCF = QVBoxLayout() #creation du layout vertical pour le curseur de flou/netteté
        self.floulabel = QLabel("Flou") #on crée un label faisant office de titre pour le curseur
        layoutCF.addWidget(self.floulabel) #on ajoute le titre au layout
        layoutCF2 = QHBoxLayout()#creation d'un nouveau layout horizontal pour le curseur et l'affichage de sa valeur
        self.curseurFlou = QSlider(Qt.Horizontal) #creation du curseur flou/netteté
        self.curseurFlou.setMinimum(0)  #on fixe sa valeur minimale a 0
        self.curseurFlou.setMaximum(100) #on fixe sa valeur maximale a 100
        self.curseurFlou.setValue(50) # on fixe sa valuer de départ à 50
        self.curseurFlou.setFixedSize(200,20) #on fixe la taille du curseur
        layoutCF2.addWidget(self.curseurFlou)  #on ajoute le curseur au layout
        #label etat du curseur flou/netteté
        self.pourcentFlou = QLabel('50%') #on initialise la valeur du label a 50%
        self.pourcentFlou.setFixedSize(70,40) #on fixe la taille de label
        self.pourcentFlou.setFrameStyle(QFrame.Panel | QFrame.Plain)  #on modifie le style d'affiche en lui donnant un contour
        layoutCF2.addWidget(self.pourcentFlou, Qt.AlignCenter)  #on ajoute le label au layout horizontal
        layoutCF.addLayout(layoutCF2)  #on ajoute le layout horizontal au layout vertical
        self.curseurFlou.valueChanged.connect(self.valuechangeFlou) #on connecte le curseur à la fonction qui s'applique lorsque le curseur est déplacé
        layout3.addLayout(layoutCF)  #on ajoute le layout vertical de zoom au layout 3
        
        
        #layout 4 : filtres négatif, detection des contours, lissage, médian, niveau de gris
        layout4 = QHBoxLayout() #on crée un layout horizontal qui prendra les 5 layouts : négatif, detection des contours, lissage, médian, niveau de gris
        layoutNeg = QVBoxLayout() #creation du layout vertical négatif
        
        #image exemple utlisé pour démontrer les filtres en attendant qu'une image soit importée
        image = Image.open('lapin_mignon.jpg') 
        #bouton filtre negatif
        self.buttonNeg = PicButton(pil2pixmap(ImageOps.invert(image)))#on crée une image cliquable qui montre l'utilisation du filtre négatif
        self.buttonNeg.clicked.connect(self.btn_Neg) #on connecte le bouton à la fonction qu'il appelle lorsqu'il est cliqué
        NegLabel = QLabel('Négatif') # on ajoute un label faisant office de titre 
        NegLabel.setAlignment(Qt.AlignCenter)#le label est aligné au centre du layout 
        layoutNeg.addWidget(self.buttonNeg) #on ajoute le bouton au layout
        layoutNeg.addWidget(NegLabel) #on ajoute le label au layout
        
        #bouton filtre detection des contours (sobel)
        layoutSob = QVBoxLayout() #creation du layout vertical detection des contours
        self.buttonSobel = PicButton(pil2pixmap(image.filter(ImageFilter.FIND_EDGES))) #on crée une image cliquable qui montre l'utilisation du filtre de détection des contours
        self.buttonSobel.clicked.connect(self.btn_Sob) #on connecte le bouton à la fonction qu'il appelle lorsqu'il est cliqué
        SobLabel = QLabel('Détéction des contours')  # on ajoute un label faisant office de titre 
        SobLabel.setAlignment(Qt.AlignCenter) #le label est aligné au centre du layout
        layoutSob.addWidget(self.buttonSobel)#on ajoute le bouton au layout
        layoutSob.addWidget(SobLabel)#on ajoute le label au layout
        
        #bouton filtre lissage
        layoutMoy = QVBoxLayout() #creation du layout vertical lissage
        self.buttonMoy = PicButton(pil2pixmap(image.filter(ImageFilter.SMOOTH))) #on crée une image cliquable qui montre l'utilisation du filtre lissant
        self.buttonMoy.clicked.connect(self.btn_Moy) #on connecte le bouton à la fonction qu'il appelle lorsqu'il est cliqué
        MoyLabel = QLabel('Lissage') # on ajoute un label faisant office de titre 
        MoyLabel.setAlignment(Qt.AlignCenter)#le label est aligné au centre du layout
        layoutMoy.addWidget(self.buttonMoy) #on ajoute le bouton au layout
        layoutMoy.addWidget(MoyLabel) #on ajoute le label au layout
        
        #bouton filtre median 
        layoutMed = QVBoxLayout()
        self.buttonMed = PicButton(pil2pixmap(image.filter(MedianFilter()))) #on crée une image cliquable qui montre l'utilisation du filtre médian
        self.buttonMed.clicked.connect(self.btn_Med) #on connecte le bouton à la fonction qu'il appelle lorsqu'il est cliqué
        MedLabel = QLabel('Médian') # on ajoute un label faisant office de titre 
        MedLabel.setAlignment(Qt.AlignCenter)#le label est aligné au centre du layout
        layoutMed.addWidget(self.buttonMed) #on ajoute le bouton au layout
        layoutMed.addWidget(MedLabel) #on ajoute le label au layout
        
        #bouton filtre niveau de gris
        layoutNG = QVBoxLayout()
        self.buttonNivGris = PicButton(pil2pixmap(image.convert('L'))) #on crée une image cliquable qui montre l'utilisation du filtre niveaux de gris
        self.buttonNivGris.clicked.connect(self.btn_NG) #on connecte le bouton à la fonction qu'il appelle lorsqu'il est cliqué
        NGLabel = QLabel('Niveau de gris') # on ajoute un label faisant office de titre 
        NGLabel.setAlignment(Qt.AlignCenter)#le label est aligné au centre du layout
        layoutNG.addWidget(self.buttonNivGris) #on ajoute le bouton au layout
        layoutNG.addWidget(NGLabel) #on ajoute le label au layout
        
        layout4.addLayout(layoutNeg) #on ajoute le layout négatif au layout4
        layout4.addLayout(layoutSob) #on ajoute le layout detection des contours au layout4
        layout4.addLayout(layoutMoy) #on ajoute le layout lissant au layout4
        layout4.addLayout(layoutMed) #on ajoute le layout médian au layout4
        layout4.addLayout(layoutNG) #on ajoute le layout niveaux de gris au layout4
        
        #layout 5 : bouton enregister l'image
        layout5 = QHBoxLayout() #on crée un layout horizontal
        self.btnEnregistre = QPushButton("Enregistrer l'image") #creation du boutn d'enregistrement
        self.btnEnregistre.setFixedSize(400,70) # on fixe la taille du bouton
        self.btnEnregistre.clicked.connect(self.btn_Enregistre) # on connecte le bouton à la fonction qu'il appelle lorqu'il est cliqué
   
        
        layout5.addWidget(self.btnEnregistre) #on ajoute le bouton au layout 5
        layout5.setAlignment(Qt.AlignRight) #les widgets du layout5 sont alignés sur la droite
        
        
        
        #ajout de tous les sous layout au layout 1
        layout1.addLayout(layout2)
        layout1.addLayout(layoutimg)
        layout1.addLayout(layout3)
        layout1.addLayout(layout4)
        layout1.addLayout(layout5)
        
        #espace entre les layout
        layout1.setSpacing(75)
        #application du layout1 a la fenetre
        self.setLayout(layout1)

    #fonction liée au curseur de zoom
    def valuechangeZoom(self):
        self.imgRetour = self.imgCurrent.copie() #image retour prend une copie de l'image courante
        imgAffiche = self.imgCurrent.copie() #image afiche prend une copie de l'image courante
        #si le curseur contraste est différent de 50 on applique la valeur du contraste a image affiche
        if (self.sizeContraste != 50):
            imgAffiche = imgAffiche.Contrast((self.sizeContraste-49)/10)
        #si le curseur flou est différent de 50 on applique la valeur du flou/netteté a image affiche
        if (self.sizeflou != 50):
            if (self.sizeflou > 50):
                imgAffiche = imgAffiche.Sharpen((self.sizeflou-49)/10)
            else :
                imgAffiche = imgAffiche.Fuzzy((51-self.sizeflou)/10)
        #on recupère la valeur du curseur zoom
        self.sizezoom = self.curseurZoom.value()
        #on change la valeur du label indiquant la valeur du curseur zoom
        self.pourcentZoom.setText(str(self.sizezoom)+'%')
        #on affiche l'image zoomée
        self.cadre.setPixmap(pil2pixmap(imgAffiche.Zoom(self.sizezoom/50).image)) 
        self.valzoom = 1 #la dernière action est le zoom donc on l'incrémente à un
        #on remet à 0 celle du contraste et du flou
        self.valconstraste = 0
        self.valflou = 0
        
        
    # fonction liée au curseur de contraste
    def valuechangeContraste(self):
        self.imgRetour = self.imgCurrent.copie() #image retour prend une copie de l'image courante
        self.sizeContraste = self.curseurContraste.value() #on recupère la valeur du curseur contraste
        imgAffiche = self.imgCurrent.copie() #image afiche prend une copie de l'image courante
        #si le curseur zoom est différent de 50 on applique la valeur du zoom a image affiche
        if (self.sizezoom != 50):
            imgAffiche = imgAffiche.Zoom(self.sizezoom/50)
        #si le curseur flou est différent de 50 on applique la valeur du flou/netteté a image affiche
        if (self.sizeflou != 50):
            if (self.sizeflou > 50):
                imgAffiche = imgAffiche.Sharpen((self.sizeflou-49)/10)
            else :
                imgAffiche = imgAffiche.Fuzzy((51-self.sizeflou)/10)
        #on change la valeur du label indiquant la valeur du curseur contraste
        self.pourcentContraste.setText(str(self.sizeContraste)+'%')
        #on affiche l'image avec le changement de contraste
        self.cadre.setPixmap(pil2pixmap(imgAffiche.Contrast((self.sizeContraste-49)/10).image))
        self.valconstraste = 1 #la dernière action est le contraste donc on l'incrémente à un
        #on remet à 0 celle du zoom et du flou
        self.valzoom = 0 
        self.valflou = 0
    
    #fonction liée au curseur flou
    def valuechangeFlou(self):
        self.imgRetour = self.imgCurrent.copie() #image retour prend une copie de l'image courante
        self.sizeflou = self.curseurFlou.value() #on recupère la valeur du curseur flou
        imgAffiche = self.imgCurrent.copie() #image afiche prend une copie de l'image courante
        #si le curseur zoom est différent de 50 on applique la valeur du zoom a image affiche
        if (self.sizezoom != 50):
            imgAffiche = imgAffiche.Zoom(self.sizezoom/50)
        #si le curseur contraste est différent de 50 on applique la valeur du contraste a image affiche
        if (self.sizeContraste != 50):
            imgAffiche = imgAffiche.Contrast((self.sizeContraste-49)/10)
        #on change la valeur du label indiquant la valeur du curseur flou
        self.pourcentFlou.setText(str(self.sizeflou)+'%')
        #si la valeur de size flou est inférieur à 50, on affiche l'image en appliquant le filtre flouteur
        if (self.sizeflou > 50):
            self.cadre.setPixmap(pil2pixmap(imgAffiche.Sharpen((self.sizeflou-49)/10).image))
        #sinon, on affiche l'image en appliquant le filtre de netteté
        else :
            self.cadre.setPixmap(pil2pixmap(imgAffiche.Fuzzy((51-self.sizeflou)/10).image))
        self.valflou = 1 #la dernière action est le flou donc on l'incrémente à un
        #on remet à 0 celle du zoom et du contraste
        self.valzoom = 0
        self.valconstraste = 0
        
    #fonction liée au bouton ok
    def btn_ok(self):
        #si le chemin existe on importe l'image
        if os.path.exists(self.cheminImage.text())  : 
            self.imageImportee = 1 #on incrément imageImportée
            self.imgbase = CustomImage(self.cheminImage.text()) #on importe l'image dans imgbase qui nous servira pour le bouton réinitialiser
            self.imgRetour = CustomImage(self.cheminImage.text()) #on importe l'image dans imgRetour qui nous servira dans le bouton Retour
            self.imgCurrent = CustomImage(self.cheminImage.text()) #on importe l'image
            self.cadre.setPixmap(pil2pixmap(self.imgCurrent.image)) # on affiche l'image dans la fenetre
            self.cadre.setStyleSheet("background-color:none") #on enlève la couleur de fond du Qlabel cadre
            
            #transformation des images cliquables démontrant les filtres 
            imgNeg = self.imgbase.copie().Negative() #création d'une image avec le filtre négatif
            self.buttonNeg.changePic(pil2pixmap(imgNeg.image).scaled(150,150,Qt.KeepAspectRatio)) #changement de l'image pour le bouton négatif
            self.buttonNeg.update() #prise en compte des modifications
            imgSob = self.imgbase.copie().Sobel() #création d'une image avec le filtre de detection des contours
            self.buttonSobel.changePic(pil2pixmap(imgSob.image).scaled(150,150,Qt.KeepAspectRatio)) #changement de l'image pour le bouton detection des contours 
            self.buttonSobel.update() #prise en compte des modifications
            imgMed = self.imgbase.copie().Median() #création d'une image avec le filtre médian
            self.buttonMed.changePic(pil2pixmap(imgMed.image).scaled(150,150,Qt.KeepAspectRatio)) #changement de l'image pour le bouton médian
            self.buttonMed.update() #prise en compte des modifications
            imgMoy = self.imgbase.copie().Moyen() #création d'une image avec le filtre lissant
            self.buttonMoy.changePic(pil2pixmap(imgMoy.image).scaled(150,150,Qt.KeepAspectRatio)) #changement de l'image pour le bouton lissant
            self.buttonMoy.update() #prise en compte des modifications
            imgNG = self.imgbase.copie().GrayScale() #création d'une image avec le filtre niveaux de gris
            self.buttonNivGris.changePic(pil2pixmap(imgNG.image).scaled(1500,150,Qt.KeepAspectRatio)) #changement de l'image pour le bouton niveaux de gris
            self.buttonNivGris.update() #prise en compte des modifications
            
            #on modifie les Qlabel pour affiche les caractéristiques de l'image importée
            self.nom.setText('Nom : ' +str(os.path.basename(self.imgbase.path))) #affichage du nom de l'image
            self.nom.setToolTip(str(os.path.basename(self.imgbase.path))) #on ajoute une infobulle avec le nom complet de l'image s'il n'est pas visible en entier dans la fenêtre
            self.taille.setText('Taille : '+str(self.imgbase.width)+"x"+str(self.imgbase.height)) #affichage de la taille de l'image
            self.format.setText('Format : ' +str(self.imgbase.format)) #affichage du format de l'image
            
            self.curseurZoom.setValue(50) #on assure la valeur de curseur zoom à 50
            self.curseurContraste.setValue(50) #on assure la valeur de curseur contraste à 50
            self.curseurFlou.setValue(50) #on assure la valeur de flou zoom à 50
            
            #on intialise les actions sur les curseurs à 0, ces valeurs nous sont utiles pour repérer si la dernière action 
            #effectuée est sur un curseur pour le bouton Retour
            self.valzoom = 0
            self.valconstraste = 0
            self.valflou = 0
            
            #on initialise bien toute les valeurs des curseurs à 50
            self.sizeflou = 50
            self.sizezoom = 50
            self.sizeContraste = 50
            
        #sinon on affiche une fentre popup de type information qui affiche le message "Chemin Invalide"
        else : 
            CheminInvalide = QMessageBox() #creation de la fenetre pop-up
            CheminInvalide.setIcon(QMessageBox.Information) #on la crée de type information 

            CheminInvalide.setText("Chemin Invalide") #on initialise le message à "Chemin invalide"
            CheminInvalide.exec_() #on lance l'affichage de la fenetre
    
    #fonction lié au bouton Retour : ne permet que de revenir sur la dernière action effectuée 
    def btn_Retour(self):
        #si une image est importée on procède a effacer la dernière modification que l'on a effectué sur celle-ci
        #sinon il ne se passe rien
        if (self.imageImportee == 1):
            #si la denrière action est sur le curseur zoom alors on réinitialise celui-ci à 50
            if (self.valzoom == 1):
                self.curseurZoom.setValue(50)
            #si la denrière action est sur le curseur contraste alors on réinitialise celui-ci à 50
            if (self.valconstraste == 1):
                self.curseurContraste.setValue(50)
            #si la denrière action est sur le curseur flou alors on réinitialise celui-ci à 50
            if (self.valflou == 1):
                self.curseurFlou.setValue(50)
            imgAffiche = self.imgCurrent.copie() #image affiche prend une copie d'image courante
            
            #les if qui suivent permettent d'afficher les modifications apportées par les curseurs sans endommagé l'image courante
             #si le curseur zoom est différent de 50 on applique la valeur du zoom a image affiche
            if (self.sizezoom != 50):
                imgAffiche = imgAffiche.Zoom(self.sizezoom/50)
             #si le curseur contraste est différent de 50 on applique la valeur du contraste a image affiche
            if (self.sizeContraste != 50):
                imgAffiche = imgAffiche.Contrast((self.sizeContraste-49)/10)
             #si le curseur flou est différent de 50 on applique la valeur du flou a image affiche
            if (self.sizeflou != 50):
                if (self.sizeflou > 50):
                    imgAffiche = imgAffiche.Sharpen((self.sizeflou-49)/10)
                else :
                    imgAffiche = imgAffiche.Fuzzy((51-self.sizeflou)/10)
            #affichage de l'image modifié dans la fenetre    
            self.cadre.setPixmap(pil2pixmap(imgAffiche.image)) #.scaled(600,300,Qt.KeepAspectRatio))
            self.imgCurrent = self.imgRetour.copie() #image courante prend une copie de l'image Retour
            
            #on remet à 0 la valeur de l'action du zoom, du contraste et du flou
            self.valzoom = 0
            self.valconstraste = 0
            self.valflou = 0

    #fonciton liée au bouton réinitialiser
    def btn_Reinit(self):
        #si une image est importée on procède a reinitialiser les curseurs et l'image courante reprend la valeur de le de l'image de base 
        #sinon il ne se passe rien
        if (self.imageImportee == 1):
            self.curseurZoom.setValue(50)# on remet le curseur zoom a 50
            self.curseurContraste.setValue(50) #on remet le curseur contraste a 50
            self.curseurFlou.setValue(50) #on remet le curseur flou à 50
            #on remet à 0 la valeur de l'action du zoom, du contraste et du flou
            self.valzoom = 0
            self.valconstraste = 0
            self.valflou = 0
            #on affiche l'image de base
            self.cadre.setPixmap(pil2pixmap(self.imgbase.image))
            self.imgCurrent = self.imgbase.copie()# l'image courante prend une copie de l'image de base
        
    def btn_Rotation(self):
        #si une image est importée on applique une rotation vers la droite à l'image courante
        #sinon il ne se passe rien
        if (self.imageImportee == 1):
            self.imgRetour = self.imgCurrent.copie() #image retour prend une copie de l'image courante
            imgAffiche = self.imgCurrent.copie() #image affiche prend une copie d'image courante
            #si le curseur zoom est différent de 50 on applique la valeur du zoom a image affiche
            if (self.sizezoom != 50):
                imgAffiche = imgAffiche.Zoom(self.sizezoom/50)
            #si le curseur contraste est différent de 50 on applique la valeur du contraste a image affiche
            if (self.sizeContraste != 50):
                imgAffiche = imgAffiche.Contrast((self.sizeContraste-49)/10)
            #si le curseur flou est différent de 50 on applique la valeur du flou a image affiche
            if (self.sizeflou != 50):
                if (self.sizeflou > 50):
                    imgAffiche = imgAffiche.Sharpen((self.sizeflou-49)/10)
                else :
                    imgAffiche = imgAffiche.Fuzzy((51-self.sizeflou)/10)
            imgAffiche = imgAffiche.RightRotation() #on applique la rotation a image affiche
            imgAffiche.image.thumbnail((600,300)) #on ajuste image affiche pour qu'elle puisse etre vu en entier dans la fenetre
            self.cadre.setPixmap(pil2pixmap(imgAffiche.image)) #on affiche l'image avec une rotation à droite
            self.imgCurrent.RightRotation() #on applique la rotation à l'image courante
            #on remet à 0 la valeur de l'action du zoom, du contraste et du flou
            self.valzoom = 0
            self.valconstraste = 0
            self.valflou = 0
        
    def btn_Neg(self):
        #si une image est importée on applique le filtre négatif à l'image courante
        #sinon il ne se passe rien
        if (self.imageImportee == 1):
            self.imgRetour = self.imgCurrent.copie() #image retour prend une copie de l'image courante
            imgAffiche = self.imgCurrent.copie() #image affiche prend une copie d'image courante
            #si le curseur zoom est différent de 50 on applique la valeur du zoom a image affiche
            if (self.sizezoom != 50):
                imgAffiche = imgAffiche.Zoom(self.sizezoom/50)
            #si le curseur contraste est différent de 50 on applique la valeur du contraste a image affiche
            if (self.sizeContraste != 50):
                imgAffiche = imgAffiche.Contrast((self.sizeContraste-49)/10)
            #si le curseur flou est différent de 50 on applique la valeur duflou a image affiche
            if (self.sizeflou != 50):
                if (self.sizeflou > 50):
                    imgAffiche = imgAffiche.Sharpen((self.sizeflou-49)/10)
                else :
                    imgAffiche = imgAffiche.Fuzzy((51-self.sizeflou)/10)
            self.cadre.setPixmap(pil2pixmap(imgAffiche.Negative().image)) #on affiche l'image avec le filtre négatif
            self.imgCurrent.Negative()  #on applique le filtre négatif à l'image courante
            #on remet à 0 la valeur de l'action du zoom, du contraste et du flou
            self.valzoom = 0
            self.valconstraste = 0
            self.valflou = 0
        
    def btn_Sob(self):
        #si une image est importée on applique le filtre de detection des contours à l'image courante
        #sinon il ne se passe rien
        if (self.imageImportee == 1):
            self.imgRetour = self.imgCurrent.copie() #image retour prend une copie de l'image courante
            self.imgCurrent = self.imgCurrent.Sobel()  #on applique le filtre de detection de contours à l'image courante
            imgAffiche = self.imgCurrent.copie() #image affiche prend une copie d'image courante
            #si le curseur zoom est différent de 50 on applique la valeur du zoom a image affiche
            if (self.sizezoom != 50):
                imgAffiche = imgAffiche.Zoom(self.sizezoom/50)
            #si le curseur contraste est différent de 50 on applique la valeur du contraste a image affiche
            if (self.sizeContraste != 50):
                imgAffiche = imgAffiche.Contrast((self.sizeContraste-49)/10)
            #si le curseur flou est différent de 50 on applique la valeur du flou a image affiche
            if (self.sizeflou != 50):
                if (self.sizeflou > 50):
                    imgAffiche = imgAffiche.Sharpen((self.sizeflou-49)/10)
                else :
                    imgAffiche = imgAffiche.Fuzzy((51-self.sizeflou)/10)
            self.cadre.setPixmap(pil2pixmap(imgAffiche.image)) #on affiche l'image avec le filtre de detection des contours
            #on remet à 0 la valeur de l'action du zoom, du contraste et du flou
            self.valzoom = 0
            self.valconstraste = 0
            self.valflou = 0
    
    def btn_Moy(self):
        #si une image est importée on applique le filtre lissant à l'image courante
        #sinon il ne se passe rien
        if (self.imageImportee == 1):
            self.imgRetour = self.imgCurrent.copie() #image retour prend une copie de l'image courante
            self.imgCurrent = self.imgCurrent.Moyen()  #on applique le filtre lissant à l'image courante
            imgAffiche = self.imgCurrent.copie() #image affiche prend une copie d'image courante
            #si le curseur zoom est différent de 50 on applique la valeur du zoom a image affiche
            if (self.sizezoom != 50):
                imgAffiche = imgAffiche.Zoom(self.sizezoom/50)
            #si le curseur contraste est différent de 50 on applique la valeur du contraste a image affiche
            if (self.sizeContraste != 50):
                imgAffiche = imgAffiche.Contrast((self.sizeContraste-49)/10)
            #si le curseur flou est différent de 50 on applique la valeur du flou a image affiche
            if (self.sizeflou != 50):
                if (self.sizeflou > 50):
                    imgAffiche = imgAffiche.Sharpen((self.sizeflou-49)/10)
                else :
                    imgAffiche = imgAffiche.Fuzzy((51-self.sizeflou)/10)
            self.cadre.setPixmap(pil2pixmap(imgAffiche.image)) #on affiche l'image avec le filtre lissant
            #on remet à 0 la valeur de l'action du zoom, du contraste et du flou
            self.valzoom = 0
            self.valconstraste = 0
            self.valflou = 0
    
    def btn_Med(self):
        #si une image est importée on applique le filtre médian à l'image courante
        #sinon il ne se passe rien
        if (self.imageImportee == 1):
            self.imgRetour = self.imgCurrent.copie() #image retour prend une copie de l'image courante
            self.imgCurrent = self.imgCurrent.Median()  #on applique le filtre médian à l'image courante
            imgAffiche = self.imgCurrent.copie() #image affiche prend une copie d'image courante
            #si le curseur zoom est différent de 50 on applique la valeur du zoom a image affiche
            if (self.sizezoom != 50):
                imgAffiche = imgAffiche.Zoom(self.sizezoom/50)
            #si le curseur contraste est différent de 50 on applique la valeur du contraste a image affiche
            if (self.sizeContraste != 50):
                imgAffiche = imgAffiche.Contrast((self.sizeContraste-49)/10)
            #si le curseur flou est différent de 50 on applique la valeur du flou a image affiche
            if (self.sizeflou != 50):
                if (self.sizeflou > 50):
                    imgAffiche = imgAffiche.Sharpen((self.sizeflou-49)/10)
                else :
                    imgAffiche = imgAffiche.Fuzzy((51-self.sizeflou)/10)
            self.cadre.setPixmap(pil2pixmap(imgAffiche.image)) #on affiche l'image avec le filtre médian
            #on remet à 0 la valeur de l'action du zoom, du contraste et du flou
            self.valzoom = 0
            self.valconstraste = 0
            self.valflou = 0
    
    def btn_NG(self):
        #si une image est importée on applique le filtre de niveaux de gris à l'image courante
        #sinon il ne se passe rien
        if (self.imageImportee == 1):
            self.imgRetour = self.imgCurrent.copie() #image retour prend une copie de l'image courante
            self.imgCurrent = self.imgCurrent.GrayScale()  #on applique le filtrede niveaux de gris à l'image courante
            imgAffiche = self.imgCurrent.copie() #image affiche prend une copie d'image courante
            #si le curseur zoom est différent de 50 on applique la valeur du zoom a image affiche
            if (self.sizezoom != 50):
                imgAffiche = imgAffiche.Zoom(self.sizezoom/50)
            #si le curseur contraste est différent de 50 on applique la valeur du contraste a image affiche
            if (self.sizeContraste != 50):
                imgAffiche = imgAffiche.Contrast((self.sizeContraste-49)/10)
            #si le curseur flou est différent de 50 on applique la valeur du flou a image affiche
            if (self.sizeflou != 50):
                if (self.sizeflou > 50):
                    imgAffiche = imgAffiche.Sharpen((self.sizeflou-49)/10)
                else :
                    imgAffiche = imgAffiche.Fuzzy((51-self.sizeflou)/10)
            self.cadre.setPixmap(pil2pixmap(imgAffiche.image)) #on affiche l'image avec le filtre de niveaux de gris
            #on remet à 0 la valeur de l'action du zoom, du contraste et du flou
            self.valzoom = 0
            self.valconstraste = 0
            self.valflou = 0
        
    def btn_Enregistre(self):
        #si une image est importée on peut enregistrer l'image
        #sinon il ne se passe rien
        if (self.imageImportee == 1):
            #si le curseur zoom est différent de 50 on applique la valeur du zoom a image courante
            if (self.sizezoom != 50):
                self.imgCurrent.Zoom(self.sizezoom/50)
            #si le curseur contraste est différent de 50 on applique la valeur du contraste a image courante
            if (self.sizeContraste != 50):
                self.imgCurrent.Contrast((self.sizeContraste-49)/10)
            #si le curseur flou est différent de 50 on applique la valeur du flou a image courante
            if (self.sizeflou != 50):
                if (self.sizeflou > 50):
                    self.imgCurrent.Sharpen((self.sizeflou-49)/10)
                else :
                    self.imgCurrent.Fuzzy((51-self.sizeflou)/10)
            #on appelle le constructeur de la fenetre de dialogue en mettant en argument l'image modifiée
            fenEnr = FenetreEnregistre(self.imgCurrent)
            #on execute la fenetre de dialogue pour enregistrer l'image
            if fenEnr.exec_() : 
                print("yeah") #pour tester si la fenêtre s'execute bien
            fenEnr.show()

#on vérifie s'il existe déjà une instance de QApplication    
app = QApplication.instance() 
# sinon on crée une instance de QApplication
if not app:
    app = QApplication(sys.argv)

#on crée une fenetre principale
fen = Fenetre()
#on l'affiche
fen.show()

#execution de l'application
app.exec_()

0